In [1]:
import pandas as pd 
import numpy as np

In [3]:
df = pd.read_csv('Data/U.S._Chronic_Disease_Indicators__CDI_.csv')

/Users/humphreyhan/Library/Caches/pypoetry/virtualenvs/mds-G2ZMO4Sf-py3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Selection of Data and Reshaping the Data

### Question 1
Remove all columns you do not need for the analysis (All done in Python, of course. No Excel acrobatics.). We are interested in two sets of variables. Select the following variables and remove all others:
a) Binge Drinking: Binge drinking prevalence among adults aged >= 18 years, Crude Prevalence in Percent. We would like to obtain this variable for the overall population, as well separately for females and males.
b) Poverty: Poverty, Crude Prevalence in Percent. We only want the overall poverty prevalence to make things a bit easier.

In [5]:
# select two sets of variables
df1 = df[df['DataValueType']=='Crude Prevalence']
df1 = df1[
    ((df1['Question'] == 'Poverty') & (df1['Stratification1'] == 'Overall'))|
    ((df1['Question'] == 'Binge drinking prevalence among adults aged >= 18 years') & (df1['Stratification1'] == 'Overall'))|
    ((df1['Question'] == 'Binge drinking prevalence among adults aged >= 18 years') & (df1['Stratification1'] == 'Female'))|
    ((df1['Question'] == 'Binge drinking prevalence among adults aged >= 18 years') & (df1['Stratification1'] == 'Male'))]

In [6]:
#remove unrelated columns
df1 = df1.iloc[:,[0,1,2,3,6,10,17]]
# test whether YearStart always equals YearEnd
# when they are equal, reserve only one
if df1['YearStart'].equals(df1['YearEnd']): 
    df1 = df1.iloc[:,1:]

In [7]:
df1.head()

,YearEnd,LocationAbbr,LocationDesc,Question,DataValue,Stratification1
340,2015,AL,Alabama,Poverty,18.5,Overall
369,2014,TX,Texas,Poverty,17.2,Overall
379,2011,NM,New Mexico,Poverty,21.5,Overall
473,2011,WY,Wyoming,Poverty,11.3,Overall
489,2011,ND,North Dakota,Poverty,12.2,Overall


### Question 2
Convert the dataset to a wide format data set using the commands from the pandas package.

In [9]:
df_wide = df1.pivot(index = ['LocationDesc','LocationAbbr','YearEnd'],columns=['Question','Stratification1'], values='DataValue')
df_wide.head(5)

Question                          Poverty  \
Stratification1                   Overall   
LocationDesc LocationAbbr YearEnd           
Alabama      AL           2010         19   
                          2011         19   
                          2012         19   
                          2013       18.7   
                          2014       19.3   

Question                          Binge drinking prevalence among adults aged >= 18 years  \
Stratification1                                                                    Female   
LocationDesc LocationAbbr YearEnd                                                           
Alabama      AL           2010                                                   NaN        
                          2011                                                   7.9        
                          2012                                                     7        
                          2013                                                   6.5        
                          2014                                                   6.8        

Question                                         
Stratification1                    Male Overall  
LocationDesc LocationAbbr YearEnd                
Alabama      AL           2010      NaN     NaN  
                          2011     20.2    13.7  
                          2012     18.2    12.3  
                          2013     16.3    11.2  
                          2014     17.9      12

### Question 3
Rename the variables to follow the format below.
Your dataset should now be in a wide state-year format with the following variables:

state: Name of the State
stateabb: State Abbreviation
year: year of observation
binge_all: Binge drinking prevalence among all adults aged >= 18 years
binge_male: Binge drinking prevalence among male adults aged >= 18 years
binge_female: Binge drinking prevalence among female adults aged >= 18 years
poverty: Poverty, Crude Prevalence in Percent
Provide an overview of the dataset by printing its size (using the shape command) and some summary statistics (using the describe command).

Save the cleaned dataset as binge_clean.csv. That file should be included in the uploaded files for your homework submission.



In [10]:
binge_clean = df_wide.reset_index()
binge_clean.columns = ['state','stateabb','year','poverty','binge_male','binge_female','binge_all']

In [11]:
binge_clean.shape

(548, 7)

In [12]:
#change datatype of binge and poverty
binge_clean[['binge_all','binge_male','binge_female','poverty']] = binge_clean[['binge_all','binge_male','binge_female','poverty']].apply(pd.to_numeric)
binge_clean.describe()

,year,poverty,binge_male,binge_female,binge_all
count,548.000000,477.000000,486.000000,486.000000,486.000000
mean,2014.516423,14.935220,11.761111,22.474280,16.968313
std,2.867218,5.227848,2.838002,3.729218,3.116518
min,2010.000000,7.300000,5.000000,13.100000,9.600000
25%,2012.000000,11.800000,9.900000,20.200000,15.100000
50%,2015.000000,14.000000,11.800000,22.100000,16.800000
75%,2017.000000,16.900000,13.200000,24.300000,18.700000
max,2019.000000,46.200000,22.300000,33.800000,27.200000


In [13]:
binge_clean.to_csv('Data/binge_clean.csv',index = False)

### Question 4
Produce a table 
that shows the overall, female, and male binge drinking prevalences across U.S. States 
in the most recent year of data 
for the Top 10 binge drinking states (i.e. the ones with the highest prevalence in the overall population). 

Use the relevant pandas commands to select the right variables, sort the data, and filter the data frame.

In [14]:
#select the most recent year
binge_2019 = binge_clean.loc[binge_clean.groupby('state')['year'].idxmax()]
#fill missing data
binge_2019['binge_all'] = binge_2019['binge_all'].fillna(-1)

In [15]:
#select the top ten
binge_top = binge_2019.sort_values('binge_all',ascending=False).head(10)
binge_top

,state,stateabb,year,poverty,binge_male,binge_female,binge_all
89,District of Columbia,DC,2019,NaN,22.3,27.8,24.9
358,North Dakota,ND,2019,NaN,14.9,29.2,22.2
537,Wisconsin,WI,2019,NaN,16.5,27.7,21.9
168,Iowa,IA,2019,NaN,16.5,27.0,21.6
438,South Dakota,SD,2019,NaN,16.1,26.4,21.2
288,Nebraska,NE,2019,NaN,15.7,26.1,20.9
248,Minnesota,MN,2019,NaN,16.3,25.3,20.7
278,Montana,MT,2019,NaN,14.7,26.7,20.7
148,Illinois,IL,2019,NaN,14.4,25.8,19.9
228,Massachusetts,MA,2019,NaN,15.4,23.2,19.1


### Question 5
Calculate the average annual growth rates (in percent) of overall binge drinking across states for the years the data is available. One way to get these growth rates, is to group the data by state (groupby) and use the first() and last() commands to get the first and last non-NA percentage followed by dividing the calculated percentage increase by the number of years data is available for. Alternatively, you could use the pct_change function to help you out. Provide a table of the 5 states with the largest increases and the 5 states with the largest decreases in binge drinking prevalence over the time period.



In [16]:
#slice related columns
binge_change = binge_clean[['state','year','binge_all']]
binge_change.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548 entries, 0 to 547
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   state      548 non-null    object 
 1   year       548 non-null    int64  
 2   binge_all  486 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.0+ KB


In [17]:
#mutate a new column which gives the perecentage of change every year
binge_change['change'] = binge_change['binge_all'].pct_change()

<ipython-input-17-4b61f83382f8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binge_change['change'] = binge_change['binge_all'].pct_change()


In [18]:
#create a dataframe which gives the average change per year
aveChange = pd.DataFrame(binge_change.groupby('state')['change'].mean().reset_index(name = 'ave_change'))

In [19]:
FiveLargestIncrease = aveChange.sort_values('ave_change',ascending = False).head(5)
FiveLargestIncrease

,state,ave_change
53,Wisconsin,0.087362
35,North Dakota,0.069755
48,Vermont,0.066150
1,Alaska,0.047275
8,District of Columbia,0.046596


In [20]:
FiveLargestDecrease = aveChange.sort_values('ave_change').head(5)
FiveLargestDecrease

,state,ave_change
9,Florida,-0.038721
25,Mississippi,-0.035410
47,Utah,-0.033386
37,Oklahoma,-0.028773
40,Puerto Rico,-0.023219
